In [1]:
! python  --version
! pip install requests
! pip install pprint
! pip install pandas

Python 3.7.3
     |████████████████████████████████| 10.4MB 22.7MB/s eta 0:00:01   |█                               | 327kB 3.3MB/s eta 0:00:04     |█████▏                          | 1.7MB 3.3MB/s eta 0:00:03     |██████▋                         | 2.2MB 3.3MB/s eta 0:00:03     |█████████▍                      | 3.1MB 3.3MB/s eta 0:00:03     |██████████▍                     | 3.4MB 3.3MB/s eta 0:00:03     |██████████████████████████▊     | 8.7MB 22.7MB/s eta 0:00:01     |████████████████████████████▏   | 9.2MB 22.7MB/s eta 0:00:01
     |████████████████████████████████| 512kB 15.7MB/s eta 0:00:01
     |████████████████████████████████| 20.0MB 424kB/s  eta 0:00:01   |████▍                           | 2.8MB 5.5MB/s eta 0:00:04     |█████▎                          | 3.3MB 5.5MB/s eta 0:00:04     |██████▏                         | 3.8MB 5.5MB/s eta 0:00:03     |███████▌                        | 4.7MB 5.5MB/s eta 0:00:03     |██████████▉                     | 6.8MB 5.5MB/s eta 0:00:03     |█

In [2]:
import requests
import icdcQueries as icdc #This is a file with various GraphQL queries
import pprint
import pandas as pd

In [3]:
#General query interface
def runQuery(url, query, variables):
    
    request = requests.post(url, json={'query': query, 'variables': variables})
    if request.status_code == 200:
        return request.json()
    else:
        raise Exception("Query failed code {}. {}".format(request.status_code,query))
    

In [4]:
def flattenJSON(jsondata):
    #https://stackoverflow.com/questions/37706351/nested-json-to-csv-generic-approach
    #https://towardsdatascience.com/flattening-json-objects-in-python-f5343c794b10?gi=4fbbb7be6da5#.y46j8vr6ghttps://medium.com/@amirziai/flattening-json-objects-in-python-f5343c794b10#.y46j8vr6g
    flatdata = {}

    def flatten(x, name=''):
        if type(x) is dict:
            for a in x:
                flatten(x[a], name + a + '_')
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, name + str(i) + '_')
                i +=1
        else:
            flatdata[name] = x

    flatten(jsondata)
    return flatdata

In [5]:
def single():
    #Production endpoint
    url = "https://caninecommons.cancer.gov/v1/graphql/"
    #Demo case
    caseid = "NCATS-COP01CCB080018"
    #Load the query file
    icdc.init()
    
    #Variables for the query have to be passed as a dictionary
    variables = {'case': caseid}
    pprint.pprint(variables)
    
    data = runQuery(url, icdc.prod_sbg_single_case, variables)
    
    pprint.pprint(data)

In [6]:
single()

{'case': 'NCATS-COP01CCB080018'}
{'data': {'case': [{'case_id': 'NCATS-COP01CCB080018',
                    'cohort': {'cohort_description': 'Melanoma',
                               'cohort_dose': '',
                               'study_arm': None},
                    'demographic': {'breed': 'Doberman Pinscher',
                                    'neutered_indicator': 'Yes',
                                    'patient_age_at_enrollment': 9,
                                    'sex': 'Female',
                                    'weight': '43.2'},
                    'diagnoses': [{'concurrent_disease': True,
                                   'concurrent_disease_type': 'lick granuloma '
                                                              'that is chronic '
                                                              'on right hind '
                                                              'leg being '
                                                             

In [12]:
def allCases():
    url = "https://caninecommons.cancer.gov/v1/graphql/"
    icdc.init()
    variables = None
    
    #Run the GraphQL
    jsondata = runQuery(url, icdc.sbg_all_cases, variables)
    
    #Flatten the JSON and push it into a dataframe
    finaldata = pd.DataFrame(flattenJSON(case) for case in jsondata['data']['case'])
    
    #Preview
    pprint.pprint(finaldata.head())
    pprint.pprint(finaldata.tail())

In [13]:
allCases()

       case_id_ cohort_cohort_description_ cohort_cohort_dose_  \
0  COTC007B0201     NSC 725776; 3mg/m2/day          3mg/m2/day   
1  COTC007B0501     NSC 725776; 3mg/m2/day          3mg/m2/day   
2  COTC007B0901     NSC 743400; 8mg/m2/day          8mg/m2/day   
3  COTC007B0502     NSC 725776; 3mg/m2/day          3mg/m2/day   
4  COTC007B0503     NSC 725776; 3mg/m2/day          3mg/m2/day   

  cohort_study_arm_study_clinical_study_designation_  \
0                                           COTC007B   
1                                           COTC007B   
2                                           COTC007B   
3                                           COTC007B   
4                                           COTC007B   

         cohort_study_arm_study_clinical_study_name_ cohort_study_arm_arm_  \
0  Preclinical Comparison of Three Indenoisoquino...            NSC 725776   
1  Preclinical Comparison of Three Indenoisoquino...            NSC 725776   
2  Preclinical Comparison of Thr